In [25]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import io
import time

In [26]:
expenditures_2020 = pd.read_csv('independent_expenditure_2020.csv', low_memory=False)
expenditures_2020

,candidate_id,candidate_name,sub_id,spe_nam,election_type,cand_office_state,cand_office_district,can_office,cand_pty_aff,expenditure_amount,...,category_code_full,payee_name,file_number,amndt_ind,transaction_id,image_number,receipt_dat,report_year,prev_file_num,dissemination_date
0,NaN,"Biden, Joseph",C90011826,/voces de la frontera action,G,WI,0,P,NaN,35000.00,...,Phone bankers for federal candidate,Good Works Matters,1444283,N,F57.000001,2.020100e+17,7-Oct-20,2020,NaN,NaN
1,P80000722,"BIDEN, JOSEPH R JR",C00348540,1199 SERVICE EMPLOYEES INT'L UNION FEDERAL POL...,G,NaN,0,P,DEMOCRATIC PARTY,85262.15,...,Digital ads,A/B Partners,1445692,N,SE.4151,2.020100e+17,9-Oct-20,2020,NaN,8-Oct-20
2,P80000722,"BIDEN, JOSEPH R JR",C00348540,1199 SERVICE EMPLOYEES INT'L UNION FEDERAL POL...,G,NaN,0,P,DEMOCRATIC PARTY,80000.00,...,Projected phone bank costs,"BERLIN ROSEN, LTD",1466866,N,SE.24651,2.020100e+17,26-Oct-20,2020,NaN,25-Oct-20
3,P80000722,"BIDEN, JOSEPH R JR",C00348540,1199 SERVICE EMPLOYEES INT'L UNION FEDERAL POL...,G,NaN,0,P,DEMOCRATIC PARTY,3742.00,...,Mailer,CENTURY DIRECT LLC,1466866,N,SE.24650,2.020100e+17,26-Oct-20,2020,NaN,25-Oct-20
4,P80000722,"BIDEN, JOSEPH R JR",C00348540,1199 SERVICE EMPLOYEES INT'L UNION FEDERAL POL...,G,NaN,0,P,DEMOCRATIC PARTY,48093.24,...,Digital ads,AB PARTNERS PBC,1470736,N,SE.24657,2.020110e+17,1-Nov-20,2020,NaN,31-Oct-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79612,NaN,"Trump, Donald J.",C00711861,bpolitical PAC,G,NaN,0,P,NaN,732.46,...,Carey Account - Email,Mailchimp,1442865,N,EDT.E.12,2.020100e+17,2-Oct-20,2020,NaN,29-Sep-20
79613,NaN,"Trump, Donald J.",C00711861,bpolitical PAC,G,NaN,0,P,NaN,6606.47,...,Carey Account - Social Media Ad Buy,Propeller,1442865,N,EDT.E.13,2.020100e+17,2-Oct-20,2020,NaN,8-Jul-20
79614,NaN,"CASTEN, SEAN",C90018094,NaN,G,IL,6T,H,NaN,50.00,...,RENTAL OF MEETING SPACE,NaN,1314809,N,F5702061932026,2.019020e+17,6-Feb-19,2020,NaN,NaN
79615,NaN,"CASTEN, SEAN",C90018094,NaN,G,IL,6T,H,NaN,250.00,...,CHOCOLATES FOR EVENT,NaN,1314809,N,F5702061932025,2.019020e+17,6-Feb-19,2020,NaN,NaN


In [27]:
columns = ['report_year', 'image_number', 'file_number', 'payee_name', 'expenditure_date', 'dissemination_date', 'expenditure_amount', 'category_code_full', 'support_oppose_indicator', 'candidate_id', 'candidate_name', 'cand_office_state', 'cand_office_district', 
'election_type', 'sub_id']

In [28]:
expenditures_2020 = expenditures_2020[columns]
expenditures_2020

,report_year,image_number,file_number,payee_name,expenditure_date,dissemination_date,expenditure_amount,category_code_full,support_oppose_indicator,candidate_id,candidate_name,cand_office_state,cand_office_district,election_type,sub_id
0,2020,2.020100e+17,1444283,Good Works Matters,5-Oct-20,NaN,35000.00,Phone bankers for federal candidate,S,NaN,"Biden, Joseph",WI,0,G,C90011826
1,2020,2.020100e+17,1445692,A/B Partners,8-Sep-20,8-Oct-20,85262.15,Digital ads,S,P80000722,"BIDEN, JOSEPH R JR",NaN,0,G,C00348540
2,2020,2.020100e+17,1466866,"BERLIN ROSEN, LTD",15-Oct-20,25-Oct-20,80000.00,Projected phone bank costs,S,P80000722,"BIDEN, JOSEPH R JR",NaN,0,G,C00348540
3,2020,2.020100e+17,1466866,CENTURY DIRECT LLC,19-Oct-20,25-Oct-20,3742.00,Mailer,S,P80000722,"BIDEN, JOSEPH R JR",NaN,0,G,C00348540
4,2020,2.020110e+17,1470736,AB PARTNERS PBC,31-Oct-20,31-Oct-20,48093.24,Digital ads,S,P80000722,"BIDEN, JOSEPH R JR",NaN,0,G,C00348540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79612,2020,2.020100e+17,1442865,Mailchimp,29-Sep-20,29-Sep-20,732.46,Carey Account - Email,O,NaN,"Trump, Donald J.",NaN,0,G,C00711861
79613,2020,2.020100e+17,1442865,Propeller,11-Sep-20,8-Jul-20,6606.47,Carey Account - Social Media Ad Buy,O,NaN,"Trump, Donald J.",NaN,0,G,C00711861
79614,2020,2.019020e+17,1314809,NaN,NaN,NaN,50.00,RENTAL OF MEETING SPACE,S,NaN,"CASTEN, SEAN",IL,6T,G,C90018094
79615,2020,2.019020e+17,1314809,NaN,NaN,NaN,250.00,CHOCOLATES FOR EVENT,S,NaN,"CASTEN, SEAN",IL,6T,G,C90018094


In [29]:
expenditures_2016 = pd.read_csv('fec-independent-expenditures.csv', low_memory=False)
expenditures_2016

,committee_id,committee_name,report_year,report_type,image_number,line_number,file_number,payee_name,payee_first_name,payee_middle_name,...,link_id,sub_id,payee_prefix,payee_suffix,is_notice,memo_text,filer_prefix,filer_suffix,schedule_type,pdf_url
0,C00042366,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,2013,24.0,13962841204,NaN,873350.0,RALSTON LAPP MEDIA,NaN,NaN,...,4062620131192064388,4062620131192066082,NaN,NaN,t,NaN,NaN,NaN,SE,http://docquery.fec.gov/cgi-bin/fecimg/?139628...
1,C00042622,MASSACHUSETTS REPUBLICAN PARTY,2013,24.0,13963170774,NaN,875042.0,CAMPAIGN HOMEBANK LLC,NaN,NaN,...,4062620131192064391,4062620131192066088,NaN,NaN,t,NaN,NaN,NaN,SE,http://docquery.fec.gov/cgi-bin/fecimg/?139631...
2,C00042622,MASSACHUSETTS REPUBLICAN PARTY,2013,24.0,13963170774,NaN,875042.0,CONNECTION STRATEGY LLC,NaN,NaN,...,4062620131192064391,4062620131192066089,NaN,NaN,t,NaN,NaN,NaN,SE,http://docquery.fec.gov/cgi-bin/fecimg/?139631...
3,C00484642,SENATE MAJORITY PAC,2013,48.0,13964445087,NaN,881782.0,WATERFRONT STRATEGIES,NaN,NaN,...,4072520131194234771,4072520131194237190,NaN,NaN,t,NaN,NaN,NaN,SE,http://docquery.fec.gov/cgi-bin/fecimg/?139644...
4,C00486845,LEAGUE OF CONSERVATION VOTERS VICTORY FUND,2013,48.0,13940511212,NaN,863741.0,"TERRA STRATEGIES, LLC",NaN,NaN,...,4040220131185953329,4040220131185956806,NaN,NaN,t,NaN,NaN,NaN,SE,http://docquery.fec.gov/cgi-bin/fecimg/?139405...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413232,C90010646,FEMINIST MAJORITY,2008,48.0,28039864697,NaN,387136.0,"JIMENEZ, VIRGINIA P",NaN,NaN,...,1101520080000632744,2120220081101720317,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413233,C90010646,FEMINIST MAJORITY,2008,48.0,28039864697,NaN,387136.0,RACKSPACE MANAGED HOSTING,NaN,NaN,...,1101520080000632744,2120220081101720319,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413234,C90010646,FEMINIST MAJORITY,2008,48.0,28039864698,NaN,387136.0,APLUSNET,NaN,NaN,...,1101520080000632744,2120220081101720322,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413235,C90010646,FEMINIST MAJORITY,2008,24.0,28039910863,NaN,387138.0,FEMINIST MAJORITY,NaN,NaN,...,1110320080000637867,2120220081101720324,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280399...


In [31]:
frames = [expenditures_2020, expenditures_2016]
expenditures_2016_2020 = pd.concat(frames)
expenditures_2016_2020

,report_year,image_number,file_number,payee_name,expenditure_date,dissemination_date,expenditure_amount,category_code_full,support_oppose_indicator,candidate_id,...,filing_form,link_id,payee_prefix,payee_suffix,is_notice,memo_text,filer_prefix,filer_suffix,schedule_type,pdf_url
0,2020,2.020100e+17,1444283.0,Good Works Matters,5-Oct-20,NaN,35000.00,Phone bankers for federal candidate,S,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020,2.020100e+17,1445692.0,A/B Partners,8-Sep-20,8-Oct-20,85262.15,Digital ads,S,P80000722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020,2.020100e+17,1466866.0,"BERLIN ROSEN, LTD",15-Oct-20,25-Oct-20,80000.00,Projected phone bank costs,S,P80000722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020,2.020100e+17,1466866.0,CENTURY DIRECT LLC,19-Oct-20,25-Oct-20,3742.00,Mailer,S,P80000722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,2.020110e+17,1470736.0,AB PARTNERS PBC,31-Oct-20,31-Oct-20,48093.24,Digital ads,S,P80000722,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413232,2008,2.803986e+10,387136.0,"JIMENEZ, VIRGINIA P",2008-10-13 00:00:00,NaN,250.00,NaN,S,P80003338,...,F5,1.101520e+18,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413233,2008,2.803986e+10,387136.0,RACKSPACE MANAGED HOSTING,2008-10-05 00:00:00,NaN,105.00,NaN,S,P80003338,...,F5,1.101520e+18,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413234,2008,2.803986e+10,387136.0,APLUSNET,2008-09-15 00:00:00,NaN,74.00,NaN,S,P80003338,...,F5,1.101520e+18,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280398...
413235,2008,2.803991e+10,387138.0,FEMINIST MAJORITY,2008-10-31 00:00:00,NaN,417.00,NaN,S,P80003338,...,F5,1.110320e+18,NaN,NaN,t,NaN,NaN,NaN,SE-F57,http://docquery.fec.gov/cgi-bin/fecimg/?280399...


In [33]:
expenditures_2016_2020.to_csv('independent_expenditures_2004-2020.csv')